# Image quality ranking example

This is a simple example for using the image quality ranking functionality in MIPLIB. The principle is the same as in the [PyImageQualityRanking](https://github.com/sakoho81/pyimagequalityranking) package (1), but MIPLIB also contains some additional toys, e.g. Fourier Ring Correlation analysis (2) which makes it possible to expand the image quality calculation a little bit.

As with the PyImageQualityRanking package you can alternatively use this functionality through a command line script. You can read more about that [here](https://github.com/sakoho81/pyimagequalityranking/wiki).

---
(1) Koho, Sami, Elnaz Fazeli, John E. Eriksson, and Pekka E. Hänninen. 2016. “Image Quality Ranking Method for Microscopy.” Scientific Reports 6 (July): 28962.

(2) Koho, S. et al. Fourier ring correlation simplifies image restoration in fluorescence microscopy. Nat. Commun. 10 3103 (2019).

In [ ]:
import os
import pandas as pd
import numpy as np

import miplib.ui.cli.miplib_entry_point_options as opts

from miplib.data.io import read
import miplib.processing.image as imops
from miplib.analysis.image_quality import image_quality_ranking as imq
import miplib.data.iterators.fourier_ring_iterators as iterators

# These are just needed to download the data from Figshare if not already available
import urllib.request as dl
import zipfile

## Setup

Here I use the same command line options interface that is used in the *pyimq.main* script. Please refer to the Wiki, or ```pyimq.main --help``` for more details. 

In [ ]:
args_list = ("--use-mask --normalize-power --bin-delta=1 " 
             " --resolution-threshold-criterion=fixed --frc-curve-fit-type=smooth-spline ").split()

options = opts.get_quality_script_options(args_list)

Check that the data directory exists, and if not, download the data from Figshare

In [ ]:
path = os.path.join(os.getcwd(), "Image_Quality_Ranking_Data")

if not os.path.exists(path):
    os.mkdir(path)
    zip_path = os.path.join(path, "images.zip")
    dl.urlretrieve("https://ndownloader.figshare.com/files/20749572", zip_path)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(path)


## Run

Now we are ready to run the script. 

In [ ]:
df = imq.batch_evaluate_image_quality(path, options)

## Evaluate

The data is saved into a Pandas DataFrame. Let's first check what it looks like. There are only a few images in the dataset, so the default print will work. I would recommend *df.describe()* for anyting more involved.

In [ ]:
df

The *batch_evaluate_image_quality()* function does not normalize the numerical values to [0, 1], so you have to do that separately, if thats what you want. Here's an example for getting that done.

In [ ]:
df_norm = df.copy()
df_norm.iloc[:, 1:] = df.iloc[:, 1:].subtract(df.iloc[:, 1:].min(), axis=1)
df_norm.iloc[:, 1:] = df_norm.iloc[:, 1:].divide(df_norm.iloc[:, 1:].max(), axis=1)

## The resolution has to be flipped around to make sense. Do the same, e.g. tp get the invfSTD measure 
df_norm["Resolution"] = 1 - df_norm["Resolution"]

## Utilize

As a simple example here, I take a couple of parameters (Resolution and Spatial entropy) that are well known to correlate with image quality. Then I create a third parameter, by simply taking an average of the two. The images are clearly divided in two groups. On this normalized scale each paramter gets its maximum (best) value at 1.

In [ ]:
df_norm["Average"] = (df_norm["tEntropy"] + df_norm["Resolution"]) / 2

df_res = df_norm.sort_values(by=['Average']).reset_index(drop=True)
df_res = df_res.loc[:, ["Average", "Resolution", "tEntropy"]]
                            
df_res.plot(subplots=True, layout=(1,3), figsize=(9,3))

So let's look, what does that mean. As it turns out, the ranking separates the two detectors that the data was acquired with. 

In [ ]:
pd.set_option('display.max_colwidth', -1)

df_check = df_norm.sort_values(by=['Average']).reset_index(drop=True)
df_check = df_check.loc[:, ["Filename", "Average", "Resolution", "tEntropy"]]

df_check